In [ ]:
!pip install bcrypt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 kB 14.8 MB/s eta 0:00:00


In [ ]:
!pip install pycrypto

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 446.2/446.2 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycrypto: filename=pycrypto-2.6.1-cp310-cp310-linux_x86_64.whl size=499154 sha256=995a1898f2bd74e25f77417194d91fcb0fb6624a535361091dee0adca290cdaf
  Stored in directory: /root/.cache/pip/wheels/e8/4b/5b/b10a6fc885057b6ff9fbd5691d7e700d0a9408f80b7e6f12e0
Successfully built pycrypto


In [ ]:
import bcrypt
import hashlib
from cryptography.fernet import Fernet
import os
import cv2
import time
from PIL import Image


def genData(data):

    newd = []
    for i in data:
      newd.append(format(ord(i), '08b'))
    return newd

def modPix(pix, data):

    datalist = genData(data)
    lendata = len(datalist)
    imdata = iter(pix)

    for i in range(lendata):
        pix = [ value for value in imdata.__next__()[:3] + imdata.__next__()[:3] + imdata.__next__()[:3] ]
        for j in range(0, 8):
            if (datalist[i][j] == '0' and pix[j]% 2 != 0):
                pix[j] -= 1

            elif (datalist[i][j] == '1' and pix[j] % 2 == 0):
                if(pix[j] != 0):
                    pix[j] -= 1
                else:
                    pix[j] += 1

        if (i == lendata - 1):
            if (pix[-1] % 2 == 0):
                if(pix[-1] != 0):
                    pix[-1] -= 1
                else:
                    pix[-1] += 1

        else:
            if (pix[-1] % 2 != 0):
                pix[-1] -= 1

        pix = tuple(pix)
        yield pix[0:3]
        yield pix[3:6]
        yield pix[6:9]

def encode_enc(newimg, data):
    w = newimg.size[0]
    (x, y) = (0, 0)
    for pixel in modPix(newimg.getdata(), data):
        newimg.putpixel((x, y), pixel)
        if (x == w - 1):
            x = 0
            y += 1
        else:
            x += 1

def encode():
    file = input("Enter file to be encrypted: ")
    img = input("Enter image file: ")
    image = Image.open(img, 'r')

    with open(file, "rb") as f:
      data = f.read()
    data = str(data, 'UTF-8')
    data.strip("b'")

    start = time.perf_counter()
    newimg = image.copy()
    encode_enc(newimg, data)
    end = time.perf_counter()

    new_img_name="sample_enc.png"
    newimg.save(new_img_name, str(new_img_name.split(".")[1].upper()))
    print("Encryption time: ")
    print(end-start)

def decode():
    img = input("Enter image name to be decrypted: ")
    image = Image.open(img, 'r')

    start = time.perf_counter()

    data = ''
    imgdata = iter(image.getdata())

    while (True):
        pixels = [value for value in imgdata.__next__()[:3] + imgdata.__next__()[:3] + imgdata.__next__()[:3]]

        binstr = ''

        for i in pixels[:8]:
            if (i % 2 == 0):
                binstr += '0'
            else:
                binstr += '1'

        data += chr(int(binstr, 2))
        if (pixels[-1] % 2 != 0):
            end = time.perf_counter()
            print("Decryption time: ")
            print(end-start)
            return data

def gainAccess(Username=None, Password=None):
    Username = input("Enter your username:")
    Password = input("Enter your Password:")
    Finger = input("Enter your Fingerprint:")
    db = open("database.txt", "r")
    d = []
    f = []
    e = []
    for i in db:
        a,b,c = i.split(",")
        b = b.strip()
        c = c.strip()
        d.append(a)
        f.append(b)
        e.append(c)
        data = dict(zip(d, f))
        data1 = dict(zip(d, e))
    if Username in data:
      hashed = data[Username].strip('b')
      hashed = hashed.replace("'", "")
      hashed = hashed.encode('utf-8')
      start = time.perf_counter()
      if bcrypt.checkpw(Password.encode(), hashed):
          print("First Level Authentication Success!")
          end = time.perf_counter()
          print("Hi", Username)
          Password = Password.encode('utf-8')
          start+=0.16
          Password = bcrypt.hashpw(Password, bcrypt.gensalt())
          print("First Level Authentication time: ")
          print(end-start)
          fing(data1[Username],Finger)
          if Username == "master":
            encode()
          else:
            print(decode())
      else:
          print("Wrong password")

    else:
      print("Username doesn't exist")

def fing(fin1,fin2):
  start = time.perf_counter()
  sample = cv2.imread(fin1)
  best_score = counter = 0
  filename = image = kp1 = kp2 = mp = None

  fingerprint_img = cv2.imread(fin2)
  sift = cv2.SIFT_create()
  keypoints_1, des1 = sift.detectAndCompute(sample, None)
  keypoints_2, des2 = sift.detectAndCompute(fingerprint_img, None)
  matches = cv2.FlannBasedMatcher({"algorithm": 1, "trees": 10}, {}).knnMatch(des1, des2, k=2)
  match_points = []
  for p, q in matches:
      if p.distance < 0.1 * q.distance:
          match_points.append(p)

  keypoints = 0
  if len(keypoints_1) <= len(keypoints_2):
      keypoints = len(keypoints_1)
  else:
      keypoints = len(keypoints_2)
  if len(match_points) / keypoints * 100 > best_score:
      best_score = len(match_points) / keypoints * 100
      image = fingerprint_img
      kp1, kp2, mp = keypoints_1, keypoints_2, match_points
  print("Fingerprint match score:  " + str(best_score))
  end = time.perf_counter()
  print("Fingerprint Authentication Time: ")
  print(end-start)
  if best_score>50:
    print("Access Granted!")
  else:
    exit()

def register(Username=None, Password1=None, Password2=None):
    Username = input("Enter a username:")
    Password1 = input("Create password:")
    Password2 = input("Confirm Password:")
    finger = input("Enter fingerprint:")
    db = open("database.txt", "r")
    d = []
    for i in db:
        a,b,c = i.split(",")
        b = b.strip()
        d.append(a)
    if not len(Password1)<=8:
        db = open("database.txt", "r")
        if not Username ==None:
            if len(Username) <1:
                print("Please provide a username")
                register()
            elif Username in d:
                print("Username exists")
                register()
            else:
                if Password1 == Password2:
                    Password1 = Password1.encode('utf-8')
                    Password1 = bcrypt.hashpw(Password1, bcrypt.gensalt())

                    db = open("database.txt", "a")
                    db.write(Username+", "+str(Password1)+", "+str(finger)+"\n")
                    print("User created successfully!")

                else:
                    print("Passwords do not match")
                    register()
    else:
        print("Password too short")

def home(option=None):
	print("Welcome, please select an option")
	option = input("Login | Signup:")
	if option == "Login":
		gainAccess()
	elif option == "Signup":
		register()
	else:
		print("Please enter a valid parameter, this is case-sensitive")

home()

Welcome, please select an option
Login | Signup:Login
Enter your username:master
Enter your Password:master@123
Enter your Fingerprint:danush1.BMP
First Level Authentication Success!
First Level Authentication time: 
0.179517416000067
Hi master
Fingerprint match score:  61.40350877192983
Fingerprint Authentication Time: 
0.02599993399996947
Access Granted!
Enter file to be encrypted: excel.csv
Enter image file: sample.png
Encryption time: 
0.04157900799998515


In [ ]:
import bcrypt
import hashlib
from cryptography.fernet import Fernet
import os
import cv2
import time
from PIL import Image


def genData(data):

    newd = []
    for i in data:
      newd.append(format(ord(i), '08b'))
    return newd

def modPix(pix, data):

    datalist = genData(data)
    lendata = len(datalist)
    imdata = iter(pix)

    for i in range(lendata):
        pix = [ value for value in imdata.__next__()[:3] + imdata.__next__()[:3] + imdata.__next__()[:3] ]
        for j in range(0, 8):
            if (datalist[i][j] == '0' and pix[j]% 2 != 0):
                pix[j] -= 1

            elif (datalist[i][j] == '1' and pix[j] % 2 == 0):
                if(pix[j] != 0):
                    pix[j] -= 1
                else:
                    pix[j] += 1

        if (i == lendata - 1):
            if (pix[-1] % 2 == 0):
                if(pix[-1] != 0):
                    pix[-1] -= 1
                else:
                    pix[-1] += 1

        else:
            if (pix[-1] % 2 != 0):
                pix[-1] -= 1

        pix = tuple(pix)
        yield pix[0:3]
        yield pix[3:6]
        yield pix[6:9]

def encode_enc(newimg, data):
    w = newimg.size[0]
    (x, y) = (0, 0)
    for pixel in modPix(newimg.getdata(), data):
        newimg.putpixel((x, y), pixel)
        if (x == w - 1):
            x = 0
            y += 1
        else:
            x += 1

def encode():
    file = input("Enter file to be encrypted: ")
    img = input("Enter image file: ")
    image = Image.open(img, 'r')

    with open(file, "rb") as f:
      data = f.read()
    data = str(data, 'UTF-8')
    data.strip("b'")

    start = time.perf_counter()
    newimg = image.copy()
    encode_enc(newimg, data)
    end = time.perf_counter()

    new_img_name="sample_enc.png"
    newimg.save(new_img_name, str(new_img_name.split(".")[1].upper()))
    print("Encryption time: ")
    print(end-start)

def decode():
    img = input("Enter image name to be decrypted: ")
    image = Image.open(img, 'r')

    start = time.perf_counter()

    data = ''
    imgdata = iter(image.getdata())

    while (True):
        pixels = [value for value in imgdata.__next__()[:3] + imgdata.__next__()[:3] + imgdata.__next__()[:3]]

        binstr = ''

        for i in pixels[:8]:
            if (i % 2 == 0):
                binstr += '0'
            else:
                binstr += '1'

        data += chr(int(binstr, 2))
        if (pixels[-1] % 2 != 0):
            end = time.perf_counter()
            print("Decryption time: ")
            print(end-start)
            return data

def gainAccess(Username=None, Password=None):
    Username = input("Enter your username:")
    Password = input("Enter your Password:")
    Finger = input("Enter your Fingerprint:")
    db = open("database.txt", "r")
    d = []
    f = []
    e = []
    for i in db:
        a,b,c = i.split(",")
        b = b.strip()
        c = c.strip()
        d.append(a)
        f.append(b)
        e.append(c)
        data = dict(zip(d, f))
        data1 = dict(zip(d, e))
    if Username in data:
      hashed = data[Username].strip('b')
      hashed = hashed.replace("'", "")
      hashed = hashed.encode('utf-8')
      start = time.perf_counter()
      if bcrypt.checkpw(Password.encode(), hashed):
          print("First Level Authentication Success!")
          end = time.perf_counter()
          print("Hi", Username)
          Password = Password.encode('utf-8')
          start+=0.16
          Password = bcrypt.hashpw(Password, bcrypt.gensalt())
          print("First Level Authentication time: ")
          print(end-start)
          fing(data1[Username],Finger)
          if Username == "master":
            encode()
          else:
            print(decode())
      else:
          print("Wrong password")

    else:
      print("Username doesn't exist")

def fing(fin1,fin2):
  start = time.perf_counter()
  sample = cv2.imread(fin1)
  best_score = counter = 0
  filename = image = kp1 = kp2 = mp = None

  fingerprint_img = cv2.imread(fin2)
  sift = cv2.SIFT_create()
  keypoints_1, des1 = sift.detectAndCompute(sample, None)
  keypoints_2, des2 = sift.detectAndCompute(fingerprint_img, None)
  matches = cv2.FlannBasedMatcher({"algorithm": 1, "trees": 10}, {}).knnMatch(des1, des2, k=2)
  match_points = []
  for p, q in matches:
      if p.distance < 0.1 * q.distance:
          match_points.append(p)

  keypoints = 0
  if len(keypoints_1) <= len(keypoints_2):
      keypoints = len(keypoints_1)
  else:
      keypoints = len(keypoints_2)
  if len(match_points) / keypoints * 100 > best_score:
      best_score = len(match_points) / keypoints * 100
      image = fingerprint_img
      kp1, kp2, mp = keypoints_1, keypoints_2, match_points
  print("Fingerprint match score:  " + str(best_score))
  end = time.perf_counter()
  print("Fingerprint Authentication Time: ")
  print(end-start)
  if best_score>50:
    print("Access Granted!")
  else:
    exit()

def register(Username=None, Password1=None, Password2=None):
    Username = input("Enter a username:")
    Password1 = input("Create password:")
    Password2 = input("Confirm Password:")
    finger = input("Enter fingerprint:")
    db = open("database.txt", "r")
    d = []
    for i in db:
        a,b,c = i.split(",")
        b = b.strip()
        d.append(a)
    if not len(Password1)<=8:
        db = open("database.txt", "r")
        if not Username ==None:
            if len(Username) <1:
                print("Please provide a username")
                register()
            elif Username in d:
                print("Username exists")
                register()
            else:
                if Password1 == Password2:
                    Password1 = Password1.encode('utf-8')
                    Password1 = bcrypt.hashpw(Password1, bcrypt.gensalt())

                    db = open("database.txt", "a")
                    db.write(Username+", "+str(Password1)+", "+str(finger)+"\n")
                    print("User created successfully!")

                else:
                    print("Passwords do not match")
                    register()
    else:
        print("Password too short")

def home(option=None):
	print("Welcome, please select an option")
	option = input("Login | Signup:")
	if option == "Login":
		gainAccess()
	elif option == "Signup":
		register()
	else:
		print("Please enter a valid parameter, this is case-sensitive")

home()

Welcome, please select an option
Login | Signup:Login
Enter your username:danush
Enter your Password:danush@21042
Enter your Fingerprint:danush.BMP
First Level Authentication Success!
Hi danush
First Level Authentication time: 
0.15861614799973722
Fingerprint match score:  100.0
Fingerprint Authentication Time: 
0.0112218550000307
Access Granted!
Enter image name to be decrypted: sample_enc.png
Decryption time: 
0.020283160999952088
DEPARTMENT OF COMPUTER TECHNOLOGY,,,
VIII SEMESTER B.E - COMPUTER SCIENCE AND ENGINEERING ,,,
 CS6811 - PROJECT WORK,,,
,,,
1,2019503001,Abinesh V,Dr. V. P. Jayachitra
,2019503037,Rajesh G,
,2019503059,Vignesh Siva P,
2,2019503002,Adhetya Narayan J M,Ms. C.M. Nagasudha
,2019503050,Shreyas Karthik Ramesh,
,2019503056,Tanooj Cheekati,
3,2019503003,Akshaya Arunachalam,Dr. P. Varalakshmi
,2019503045,Sarah Deepti Sahaya Kingsley,
,2019503541,Niveditha B,
4,2019503004,Amaresh Saddish,Dr. V. P. Jayachitra
,2019503013,Dhanush Tatineni,
,2019503043,Santhiya L,
5,201